In [1]:
'''
Answer in this format 'value unit' using the text on the image, What is the item weight?
'''

"\nAnswer in this format 'value unit' using the text on the image, What is the item weight?\n"

In [2]:
import pandas as pd
import numpy as np
import json as js

import sys
sys.path.append('./student_resource 3/src')

from utils import *
from constants import *
from pathlib import Path

In [3]:
data = pd.read_csv("student_resource 3/dataset/train.csv")

In [4]:
data.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [5]:
NUM_SAMPLES = 15

In [6]:
max_len = data.shape[0]
print(max_len)

263859


In [7]:
nums = np.random.randint(0, max_len-1, NUM_SAMPLES)

In [8]:
with open("questions.jsonl", "w") as f:
    for i in nums:
        url, grp_id, e_name, e_val = data.iloc[i, :]
        f_name = Path(url).name
        download_image(url, "images")
        print(f"q{i} : {e_name} : {e_val}")
        
        dct = {"question_id": f"q{i+1}", "text": f"Answer in this format 'value unit' using the text on the image, What is the {' '.join(e_name.split('_'))}?", "image": f"{f_name}"}
        
        f.write(js.dumps(dct) + '\n')
        

q72918 : item_weight : 1190.0 gram
q163866 : width : 5.9 inch
q9470 : wattage : 150.0 watt
q183030 : depth : 120.0 millimetre
q221754 : depth : 4.6 inch
q238915 : height : 3.25 inch
q132639 : width : 28.0 inch
q252748 : height : 8.9 centimetre
q187802 : depth : 0.9 centimetre
q113815 : item_weight : 10.0 gram
q187821 : depth : 11.6 centimetre
q177226 : height : 39.0 inch
q90105 : item_weight : 18.14 gram
q58102 : item_weight : 200.0 milligram
q130774 : item_weight : 24.0 gram
